In [1]:
# load all necessary libraries
import pandas as pd
from nltk.tokenize import word_tokenize ,sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import jellyfish as jf


pd.set_option('max_colwidth', 100)

In [2]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\mirza\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\mirza\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\mirza\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\mirza\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\mirza\AppData\Roaming\nltk_data...
[

True

In [52]:
df=pd.read_csv("SMSSpamCollection.txt", sep="\t", names=["label", "message"])
df['label'] = df['label'].map({'ham': 0, 'spam': 1}) # convert labels to binary values
df['label'] = df['label'].astype(int) # convert labels to int

In [53]:
df.head()

,label,message
0,0,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there g..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives around here though"


In [54]:
sentences = df['message'].tolist() # convert messages to list of sentences

#### data

In [55]:
sentences

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
 'Ok lar... Joking wif u oni...',
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
 'U dun say so early hor... U c already then say...',
 "Nah I don't think he goes to usf, he lives around here though",
 "FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, Â£1.50 to rcv",
 'Even my brother is not like to speak with me. They treat me like aids patent.',
 "As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune",
 'WINNER!! As a valued network customer you have been selected to receivea Â£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.',
 'Had y

#### preprocess``

In [56]:
lami=WordNetLemmatizer() # initialize lemmatizer

In [57]:
w=nltk.word_tokenize("hello world") # tokenize a sample sentence
w

['hello', 'world']

In [58]:
for i in range(len(sentences)):
    words = nltk.word_tokenize(sentences[i].lower()) # tokenize each sentence into words
    words=[lami.lemmatize(word,)for word in words if word not in set(stopwords.words("english")) ]
    # words=jf.soundex(words) # convert words to soundex code
    sentences[i]=' '.join(words) # join words back into a sentence

sentences

['go jurong point , crazy .. available bugis n great world la e buffet ... cine got amore wat ...',
 'ok lar ... joking wif u oni ...',
 "free entry 2 wkly comp win fa cup final tkts 21st may 2005. text fa 87121 receive entry question ( std txt rate ) & c 's apply 08452810075over18 's",
 'u dun say early hor ... u c already say ...',
 "nah n't think go usf , life around though",
 "freemsg hey darling 's 3 week 's word back ! 'd like fun still ? tb ok ! xxx std chgs send , â£1.50 rcv",
 'even brother like speak . treat like aid patent .',
 "per request 'melle melle ( oru minnaminunginte nurungu vettam ) ' set callertune caller . press * 9 copy friend callertune",
 'winner ! ! valued network customer selected receivea â£900 prize reward ! claim call 09061701461. claim code kl341 . valid 12 hour .',
 'mobile 11 month ? u r entitled update latest colour mobile camera free ! call mobile update co free 08002986030',
 "'m gon na home soon n't want talk stuff anymore tonight , k ? 've cried en

In [69]:
from nltk.stem import RegexpStemmer

reg=RegexpStemmer("’|\.|'|‘|\?|,|—|”|!|:|;")

In [70]:
sentences=[reg.stem(sen) for sen in sentences]

In [71]:
sentences[0]


'go jurong point  crazy  available bugis n great world la e buffet  cine got amore wat '

In [72]:
process_sentences=[]
for i in range(len(sentences)):
    words = nltk.word_tokenize(sentences[i])
    words=[jf.soundex(word) for word in words]
    process_sentences.append(" ".join(words))

In [73]:
process_sentences

['G000 J652 P530 C620 A141 B220 N000 G630 W643 L000 E000 B130 C500 G300 A560 W300',
 'O200 L600 J252 W100 U000 O500',
 'F600 E536 2000 W240 C510 W500 F000 C100 F540 T232 2230 M000 2000 T230 F000 8000 R210 E536 Q235 (000 S300 T230 R300 )000 &000 C000 S000 A140 0160 S000',
 'U000 D500 S000 E640 H600 U000 C000 A463 S000',
 'N000 N300 T520 G000 U210 L100 A653 T200',
 'F652 H000 D645 S000 3000 W200 S000 W630 B200 D000 L200 F500 S340 T100 O200 X000 S300 C000 S530 A000 R210',
 'E150 B636 L200 S120 T630 L200 A300 P353',
 'P600 R223 M400 M400 (000 O600 M555 N652 V350 )000 S300 C463 C460 P620 *000 9000 C100 F653 C463',
 'W560 V430 N362 C235 S423 R210 A000 P620 R630 C450 C400 0000 C450 C300 K400 V430 1000 H600',
 'M140 1000 M530 U000 R000 E533 U133 L323 C460 M140 C560 F600 C400 M140 U133 C000 F600 0000',
 'M000 G500 N000 H500 S500 N300 W530 T420 S310 A556 T523 K000 V000 C630 E520 T300',
 'S200 C520 W500 C200 1000 2000 P530 T230 >000 C000 S530 8000 C230 1130 6320 1000 T253 A140 R140 H400 4000 I510

In [74]:
vec=TfidfVectorizer()

In [80]:
bow=vec.fit_transform(process_sentences)

In [81]:
bow.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.24180416, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.24154785, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [82]:
mdf=pd.DataFrame(bow.toarray(),columns=vec.get_feature_names_out())

In [83]:
mdf

,000,0000,0100,0155,0160,0165,0200,0215,0231,0252,...,z000,z160,z200,z236,z300,z453,z500,z520,z532,z600
0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.241804,0.000000,0.0,0.0,0.271591,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0.000000,0.168451,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5568,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5569,0.241548,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5570,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [86]:
X=mdf

In [87]:
y=df['label']

In [88]:
print(X.shape,y.shape)

(5572, 2475) (5572,)


In [89]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

In [90]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=100)

In [91]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(4179, 2475) (1393, 2475) (4179,) (1393,)


In [93]:
model=MultinomialNB()
model

MultinomialNB()

In [94]:
model.fit(X_train,y_train)

MultinomialNB()

In [95]:
y_pred=model.predict(X_test)

In [102]:
from sklearn.metrics import accuracy_score , confusion_matrix ,classification_report

In [98]:
acc=accuracy_score(y_test,y_pred)

In [99]:
acc

0.9547738693467337

In [101]:
con=confusion_matrix(y_test,y_pred)
con

array([[1206,    0],
       [  63,  124]], dtype=int64)

In [104]:
clf=classification_report(y_test,y_pred)
print(clf)

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      1206
           1       1.00      0.66      0.80       187

    accuracy                           0.95      1393
   macro avg       0.98      0.83      0.89      1393
weighted avg       0.96      0.95      0.95      1393

